In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from fastai.vision.all import *


In [ ]:
from zipfile import ZipFile 
  
# specifying the zip file name 
file_name = "/kaggle/input/aerial-cactus-identification/train.zip"
  
# opening the zip file in READ mode 
with ZipFile(file_name, 'r') as zip: 
    # printing all the contents of the zip file 
    #zip.printdir() 
  
    # extracting all the files 
    print('Extracting all the files now...') 
    zip.extractall() 
    print('Done!') 

In [ ]:
from zipfile import ZipFile 
  
# specifying the zip file name 
file_name = "/kaggle/input/aerial-cactus-identification/test.zip"
  
# opening the zip file in READ mode 
with ZipFile(file_name, 'r') as zip: 
    # printing all the contents of the zip file 
    #zip.printdir() 
  
    # extracting all the files 
    print('Extracting all the files now...') 
    zip.extractall() 
    print('Done!') 

In [ ]:
df=pd.read_csv("/kaggle/input/aerial-cactus-identification/train.csv")

In [ ]:
df.head()

In [ ]:
test_df=pd.read_csv("/kaggle/input/aerial-cactus-identification/sample_submission.csv")
test_df.head()

In [ ]:
path=Path("/kaggle/working")
path.ls()

In [ ]:
for i in df["id"]:
    path=f"/kaggle/working/train/{i}"
    isFile = os.path.isfile(path)
    if(isFile==False):
        print("Path does not exist")
        break
print("ALL ok")

In [ ]:
dblock=DataBlock(blocks=(ImageBlock,CategoryBlock),
                get_x=ColReader(0,pref=f"/kaggle/working/train/"),
                get_y=ColReader(1),
                splitter=RandomSplitter(),
                item_tfms=Resize(460),
                batch_tfms=[*aug_transforms(size=224,min_scale=0.75),Normalize.from_stats(*imagenet_stats)])

In [ ]:
test_db=ImageDataLoaders.from_df(test_df, path="/kaggle/working/test")

In [ ]:
batch_tfms=[*aug_transforms(size=224,min_scale=0.75),Normalize.from_stats(*imagenet_stats)]

In [ ]:
dblock.summary(df)

In [ ]:
dls=dblock.dataloaders(df)
dls.show_batch()

In [ ]:
x,y=dls.one_batch()
x.mean(dim=[0,2,3]),x.std(dim=[0,2,3])

In [ ]:
learn=cnn_learner(dls,resnet34,metrics=[error_rate,accuracy])

In [ ]:
learn.fit_one_cycle(5)

In [ ]:
learn.lr_find()

In [ ]:
lr=6.309573450380412e-07
learn=learn.to_fp16()


In [ ]:
learn.fit_one_cycle(5,slice(lr))

In [ ]:
learn.unfreeze()
learn.lr_find()

In [ ]:
learn.fit_one_cycle(5,slice(7.585775847473997e-07))

In [ ]:
learn.show_results(figsize=(12,12))

In [ ]:
interp=ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(2,slice(2.2908675418875646e-06))

In [ ]:
path=Path("/kaggle/working/test")
path.ls()

In [ ]:
path_img=("/kaggle/working/test/ba1c80a40755c0a0c180f254d2276653.jpg")

img = PILImage(PILImage.create(path_img).resize((460,460)))
img.shape, type(img)

In [ ]:
img

In [ ]:
def get_predict(file):
    return learn.predict(file)[2][1].item()

In [ ]:
get_predict(img)

In [ ]:
img_name=[]
predict=[]

In [ ]:
for file in os.listdir("/kaggle/working/test"):
    full_path=f"/kaggle/working/test/{file}"
    img_name.append(file)
    img = PILImage(PILImage.create(full_path).resize((224,224)))
    predict.append(get_predict(img))
    
    

In [ ]:
img_name[:5]

In [ ]:
predict[:5]

In [ ]:
dict={"id":img_name,"has_cactus":predict}

In [ ]:
df_submit=pd.DataFrame(dict)

In [ ]:
df_submit.head()

In [ ]:
df_submit.to_csv("submission.csv",index=False)